In [1]:
import numpy as np
import pandas as pd

import pystan
from sklearn.model_selection import train_test_split
import pickle
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from argparse import ArgumentParser, ArgumentTypeError
from pathlib import Path

In [2]:
class Compas_dataset():
    
    def __init__(self, data_file='COMPAS_dataset/compas-scores-two-years.csv'):
        self.cols = ['sex', 'age', 'race', 'juv_fel_count', 
                    'juv_misd_count', 'juv_other_count', 
                    'priors_count', 'c_charge_degree', 
                    'score_text', 'v_score_text', 'two_year_recid']
        self.full_df = pd.read_csv(data_file, usecols=self.cols)
        
        # Preprocessing 
        self.full_df = self.preprocessing(self.full_df)
        
        # Split train and test
        self.train_df, self.test_df = self.split_train_test(self.full_df)
        
    
    def preprocessing(self, df):
        
        # Remove rows with unknown data
        df = df.dropna()
        
        # Convert categorical to int 
        df['sex'] = df['sex'].apply(lambda x: 0 if x=='Female' else 1)
        df['c_charge_degree'] = df['c_charge_degree'].apply(lambda x: 0 if x=='M' else 1)
        df['race'] = df['race'].apply(lambda x: 0 if x=='Caucasian' else 1)

        # Normalize between 0 and 1
        df['age'] = df['age'] / df['age'].max()
        df['juv_fel_count'] = df['juv_fel_count'] / df['juv_fel_count'].max()
        df['juv_misd_count'] = df['juv_misd_count'] / df['juv_misd_count'].max()
        df['juv_other_count'] = df['juv_other_count'] / df['juv_other_count'].max()
        df['priors_count'] = df['priors_count'] / df['priors_count'].max()
        
        score_mapping = {
            "Low": 0,
            "Medium": 0.5,
            "High": 1
        }
        df['score_text'] = df['score_text'].apply(lambda x: score_mapping[x]) 
        df['v_score_text'] = df['v_score_text'].apply(lambda x: score_mapping[x]) 
        
        return df
    
    def split_train_test(self, df, train_size=0.8):
        msk = np.random.rand(len(df)) < train_size
        train_df = df[msk]
        test_df = df[~msk]
        return train_df, test_df

compas_ds = Compas_dataset()
compas_ds.full_df
compas_ds.train_df.describe()

,sex,age,race,juv_fel_count,juv_misd_count,juv_other_count,priors_count,c_charge_degree,score_text,v_score_text,two_year_recid
count,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000
mean,0.809697,0.361699,0.665628,0.003316,0.007006,0.006713,0.091615,0.649004,0.331948,0.225022,0.453853
std,0.392574,0.123715,0.471811,0.023372,0.037525,0.030711,0.127973,0.477323,0.394510,0.335735,0.497909
min,0.000000,0.187500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.260417,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.322917,1.000000,0.000000,0.000000,0.000000,0.052632,1.000000,0.000000,0.000000,0.000000
75%,1.000000,0.437500,1.000000,0.000000,0.000000,0.000000,0.131579,1.000000,0.500000,0.500000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Models

In [3]:
# wrapper class for statsmodels linear regression (more stable than SKLearn)
class SM_LinearRegression():
    def __init__(self):
        pass

    def fit(self, X, y):
        N = X.shape[0]
        self.LRFit = sm.OLS(y, np.hstack([X, np.ones(N).reshape(-1, 1)]), hasconst=True).fit()

    def predict(self, X):
        N = X.shape[0]
        return self.LRFit.predict(np.hstack([X, np.ones(N).reshape(-1, 1)]))

In [4]:
# Get the Unfair Model predictions
def Unfair_Model_Replication(train_df, test_df, target="two_year_recid"):
    lr_unfair = SM_LinearRegression()
    train_df_x = train_df.drop(columns=[target]).to_numpy()
    train_df_y = train_df[target].to_numpy()
    lr_unfair.fit(train_df_x, train_df_y)
    
    test_df_x = test_df.drop(columns=[target]).to_numpy()
    test_df_y = test_df[target].to_numpy()
    preds = lr_unfair.predict(test_df_x)

    # Return Results:
    return preds

# Get the Unaware Model predictions
def Unaware_Model_Replication(train_df, test_df, target="two_year_recid", protected=['sex', 'race']):
    lr_unfair = SM_LinearRegression()
    train_df_x = train_df.drop(columns=[target] + protected).to_numpy()
    train_df_y = train_df[target].to_numpy()
    lr_unfair.fit(train_df_x, train_df_y)
    
    test_df_x = test_df.drop(columns=[target] + protected).to_numpy()
    test_df_y = test_df[target].to_numpy()
    preds = lr_unfair.predict(test_df_x)

    # Return Results:
    return preds

In [5]:
# Get the Fair All/L3 Model Predictions
def L3_Model_Replication(train_df, test_df, target="two_year_recid", protected=['sex', 'race']):
    train_df_protected = train_df[protected].to_numpy()
    test_df_protected = test_df[protected].to_numpy()
    train_df_y = train_df[target].to_numpy()
    test_df_y = test_df[target].to_numpy()
    
    # juv_fel_count
    linear_eps_w = SM_LinearRegression()
    linear_eps_w.fit(train_df_protected, train_df['juv_fel_count'])
    eps_w_train = train_df['juv_fel_count'].to_numpy() - linear_eps_w.predict(train_df_protected)
    eps_w_test = test_df['juv_fel_count'].to_numpy() - linear_eps_w.predict(test_df_protected)
    
    # juv_misd_count
    linear_eps_e = SM_LinearRegression()
    linear_eps_e.fit(train_df_protected, train_df['juv_misd_count'])
    eps_e_train = train_df['juv_misd_count'].to_numpy() - linear_eps_e.predict(train_df_protected)
    eps_e_test = test_df['juv_misd_count'].to_numpy() - linear_eps_e.predict(test_df_protected)
    
    # juv_other_count
    linear_eps_o = SM_LinearRegression()
    linear_eps_o.fit(train_df_protected, train_df['juv_other_count'])
    eps_o_train = train_df['juv_other_count'].to_numpy() - linear_eps_o.predict(train_df_protected)
    eps_o_test = test_df['juv_other_count'].to_numpy() - linear_eps_o.predict(test_df_protected)
    
    # priors_count
    linear_eps_g = SM_LinearRegression()
    linear_eps_g.fit(train_df_protected, train_df['priors_count'])
    eps_g_train = train_df['priors_count'].to_numpy() - linear_eps_g.predict(train_df_protected)
    eps_g_test = test_df['priors_count'].to_numpy() - linear_eps_g.predict(test_df_protected)
    
    # c_charge_degree
    linear_eps_l = SM_LinearRegression()
    linear_eps_l.fit(train_df_protected, train_df['c_charge_degree'])
    eps_l_train = train_df['c_charge_degree'].to_numpy() - linear_eps_l.predict(train_df_protected)
    eps_l_test = test_df['c_charge_degree'].to_numpy() - linear_eps_l.predict(test_df_protected)
    
    # score_text
    linear_eps_h = SM_LinearRegression()
    linear_eps_h.fit(train_df_protected, train_df['score_text'])
    eps_h_train = train_df['score_text'].to_numpy() - linear_eps_h.predict(train_df_protected)
    eps_h_test = test_df['score_text'].to_numpy() - linear_eps_h.predict(test_df_protected)
    
    # v_score_text
    linear_eps_v = SM_LinearRegression()
    linear_eps_v.fit(train_df_protected, train_df['v_score_text'])
    eps_v_train = train_df['v_score_text'].to_numpy() - linear_eps_v.predict(train_df_protected)
    eps_v_test = test_df['v_score_text'].to_numpy() - linear_eps_v.predict(test_df_protected)
    

    # predict on target using abducted latents
    train_eps_stacked = np.hstack((eps_w_train.reshape(-1, 1), eps_e_train.reshape(-1, 1), 
                                   eps_o_train.reshape(-1, 1), eps_g_train.reshape(-1, 1), 
                                   eps_l_train.reshape(-1, 1), eps_h_train.reshape(-1, 1), 
                                   eps_v_train.reshape(-1, 1),
                                   train_df['age'].to_numpy().reshape(-1, 1)))
    
    test_eps_stacked = np.hstack((eps_w_test.reshape(-1, 1), eps_e_test.reshape(-1, 1), 
                                  eps_o_test.reshape(-1, 1), eps_g_test.reshape(-1, 1), 
                                  eps_l_test.reshape(-1, 1), eps_h_test.reshape(-1, 1), 
                                  eps_v_test.reshape(-1, 1),
                                  test_df['age'].to_numpy().reshape(-1, 1)))
    smlr_L3 = SM_LinearRegression()
    smlr_L3.fit(train_eps_stacked, train_df_y)

    # predict on test epsilons
    preds = smlr_L3.predict(test_eps_stacked)

    # Return Results:
    return preds

In [6]:
unfair_preds = Unfair_Model_Replication(compas_ds.train_df, compas_ds.test_df)

In [7]:
unaware_preds = Unaware_Model_Replication(compas_ds.train_df, compas_ds.test_df)

In [8]:
l3_preds = L3_Model_Replication(compas_ds.train_df, compas_ds.test_df)

In [9]:
print('Unfair RMSE: \t\t\t%.3f' % np.sqrt(mean_squared_error(unfair_preds, compas_ds.test_df['two_year_recid'])))
print('FTU RMSE: \t\t\t%.3f' % np.sqrt(mean_squared_error(unaware_preds, compas_ds.test_df['two_year_recid'])))
print('Level 3 (Fair Add) RMSE: \t%.3f' % np.sqrt(mean_squared_error(l3_preds, compas_ds.test_df['two_year_recid'])))

Unfair RMSE: 			0.455
FTU RMSE: 			0.456
Level 3 (Fair Add) RMSE: 	0.458


## Metrics

### Balanced accuracy

In [10]:
from sklearn.metrics import balanced_accuracy_score

In [11]:
target='two_year_recid'
print('Unfair balanced accuracy: \t\t%.3f' % balanced_accuracy_score(compas_ds.test_df[target], np.array(unfair_preds >= 0.5).astype(int)))
print('FTU balanced accuracy: \t\t\t%.3f' % balanced_accuracy_score(compas_ds.test_df[target], np.array(unaware_preds >= 0.5).astype(int)))
print('Level 3 (Fair Add) balanced accuracy: \t%.3f' % balanced_accuracy_score(compas_ds.test_df[target], np.array(l3_preds >= 0.5).astype(int)))

Unfair balanced accuracy: 		0.670
FTU balanced accuracy: 			0.672
Level 3 (Fair Add) balanced accuracy: 	0.659


### Equalized odds

In [12]:
preds_df = compas_ds.test_df.copy()
preds_df['unfair'] = np.array(unfair_preds >= 0.5).astype(int)
preds_df['unaware'] = np.array(unaware_preds >= 0.5).astype(int)
preds_df['l3'] = np.array(l3_preds >= 0.5).astype(int)

In [13]:
def equalized_odds(preds_df, method='unfair', target='two_year_recid', protected=['sex', 'race']):
    
    preds_df['binary_age'] = preds_df['age'] >= 0.5
    eq_odds = []
    for attribute in protected:
        eq_odd = preds_df.loc[(preds_df[target]==1) & (preds_df[attribute]==0), method].mean() -  preds_df.loc[(preds_df[target]==1) & (preds_df[attribute]==1), method].mean()
        eq_odd += preds_df.loc[(preds_df[target]==0) & (preds_df[attribute]==0), method].mean() -  preds_df.loc[(preds_df[target]==0) & (preds_df[attribute]==1), method].mean()
        eq_odds.append(np.abs(eq_odd))
    return np.mean(eq_odds)

In [14]:
print('Unfair Equalized Odds score: \t\t\t%.3f' % equalized_odds(preds_df, method='unfair'))
print('FTU Equalized Odds score: \t\t\t%.3f' % equalized_odds(preds_df, method='unaware'))
print('Level 3 (Fair Add) Equalized Odds score: \t%.3f' % equalized_odds(preds_df, method='l3'))

Unfair Equalized Odds score: 			0.341
FTU Equalized Odds score: 			0.248
Level 3 (Fair Add) Equalized Odds score: 	0.112
